# 优化模型参数
现在我们有了一个模型和数据，是时候通过优化模型参数来训练、验证和测试我们的模型了。训练模型是一个迭代过程；在每次迭代中，模型都会猜测输出，计算猜测中的误差（*损失*），收集误差相对于其参数的导数（如我们在上一节中所见），并使用梯度下降法**优化**这些参数。有关此过程的更详细介绍，请观看 3Blue1Brown 的反向传播视频。

## 先决条件代码
我们加载了之前关于数据集和数据加载器和构建模型部分的代码。

In [10]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 超参数
超参数是可调整的参数，允许您控制模型优化过程。不同的超参数值会影响模型训练和收敛速度（阅读更多关于超参数调整的信息）

我们为训练定义了以下超参数:
- **时期数** - 迭代数据集的次数

- **批量大小** - 在更新参数之前通过网络传播的数据样本数

- 学习率 - 在每个批次/时期更新模型参数的程度。较小的值会导致学习速度慢，而较大的值可能会导致训练期间出现不可预测的行为。

In [11]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 优化循环
设置超参数后，我们就可以使用优化循环来训练和优化模型。优化循环的每次迭代称为一个时期。

每个时期都包含两个主要部分
- 训练循环 - 遍历训练数据集并尝试收敛到最佳参数。

- 验证/测试循环 - 遍历测试数据集以检查模型性能是否有所提高。

## 损失函数
当提供一些训练数据时，我们未经训练的网络很可能无法给出正确的答案。损失函数衡量获得的结果与目标值的差异程度，并且是我们希望在训练期间最小化的函数。为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。

常见的损失函数包括用于回归任务的 nn.MSELoss（均方误差）和用于分类的 nn.NLLLoss（负对数似然）。nn.CrossEntropyLoss 结合了 nn.LogSoftmax 和 nn.NLLLoss。

我们将模型的输出 logits 传递给 nn.CrossEntropyLoss，它将对 logits 进行归一化并计算预测误差。

In [12]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## 优化器
优化是在每个训练步骤中调整模型参数以减少模型误差的过程。优化算法定义了此过程的执行方式（在本例中，我们使用随机梯度下降）。所有优化逻辑都封装在 optimizer 对象中。在这里，我们使用 SGD 优化器；此外，PyTorch 中还有许多不同的优化器可用，例如 ADAM 和 RMSProp，它们适用于不同类型的模型和数据。

我们通过注册需要训练的模型参数并传递学习率超参数来初始化优化器。

In [13]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在训练循环中，优化分三个步骤进行
- 调用 optimizer.zero_grad() 重置模型参数的梯度。默认情况下，梯度会累加；为了防止重复计算，我们在每次迭代时将其显式归零。

- 通过调用 loss.backward() 反向传播预测损失。PyTorch 将损失相对于每个参数的梯度存储起来。

- 获得梯度后，我们调用 optimizer.step() 根据反向传播中收集的梯度调整参数。

## 完整实现

我们定义了 train_loop 来循环执行优化代码，并定义了 test_loop 来评估模型在测试数据上的性能。

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


我们初始化损失函数和优化器，并将它们传递给 train_loop 和 test_loop。您可以随意增加时期数以跟踪模型性能的提升。

In [15]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298287  [   64/60000]


loss: 2.290331  [ 6464/60000]
loss: 2.271025  [12864/60000]
loss: 2.274470  [19264/60000]
loss: 2.258491  [25664/60000]
loss: 2.237466  [32064/60000]
loss: 2.237437  [38464/60000]
loss: 2.208437  [44864/60000]
loss: 2.200825  [51264/60000]
loss: 2.184726  [57664/60000]
Test Error: 
 Accuracy: 53.0%, Avg loss: 2.174913 

Epoch 2
-------------------------------
loss: 2.185234  [   64/60000]
loss: 2.176156  [ 6464/60000]
loss: 2.123527  [12864/60000]
loss: 2.137407  [19264/60000]
loss: 2.099551  [25664/60000]
loss: 2.045707  [32064/60000]
loss: 2.067042  [38464/60000]
loss: 1.997078  [44864/60000]
loss: 1.993707  [51264/60000]
loss: 1.931670  [57664/60000]
Test Error: 
 Accuracy: 58.6%, Avg loss: 1.929157 

Epoch 3
-------------------------------
loss: 1.966349  [   64/60000]
loss: 1.937357  [ 6464/60000]
loss: 1.825720  [12864/60000]
loss: 1.856254  [19264/60000]
loss: 1.754730  [25664/60000]
loss: 1.709135  [32064/60000]
loss: 1.721985  [38464/60000]
loss: 1.630143  [44864/60000]
loss: 